In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import random
import csv


In [7]:
#user-agent경우 구글링으로 구해면 됌
user_agent_list = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246",
"Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9",
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36",
"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1",
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36",
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36 Edg/99.0.1150.36",
"Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko",
"Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36",
"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0"
]

In [8]:
pattern = r"\s{2,}"

In [ ]:
with open('ppomppu.csv', 'w', newline='') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(['title', 'link', 'date', 'view', 'content'])

    for PAGE in range(1, 371):
        print(f'page: {PAGE} is start')

        headers = {'User-Agent': f'{random.choice(user_agent_list)}'} #여러 user-agent로 들어가기
        url = f'https://www.ppomppu.co.kr/zboard/zboard.php?id=stock&page={PAGE}&divpage=50'
        res = requests.get(url, headers=headers)

        soup = BeautifulSoup(res.text, 'lxml')
        rows = soup.find('table', {'id': 'revolution_main_table'}).find_all('tr', class_=['common-list0', 'common-list1'])#제목 가져오기

        for row in rows:
            try:
                elem = row.select('td:nth-child(3) > a')[0] #select는 css 선택할 수 있음
                #td:nth-child(3) > a: 테이블의 세 번째 셀<"td">선택. "nth-child"는 css선택자, 주어진 순번의 자식요소를 선택
                #>a: 바로 위의 선택자로 선택된 요소의 직계 자식 중에서 <a>태그를 선택->href
            except:
                continue
            title = elem.get_text(strip=True) #strip: 앞/뒤 공백, 개행문자, 택문자 제거
            link = f'https://www.ppomppu.co.kr/zboard/{elem["href"]}'

            date = row.select('td:nth-child(4)')[0].get_text(strip=True) #tr에서 4번째 td
            view = row.select('td:nth-child(6)')[0].get_text(strip=True) #tr에서 6번째 td

            headers = {'User-Agent': f'{random.choice(user_agent_list)}'}
            get_in_res = requests.get(link, headers=headers) #링크 요청, user-agent 줌 후 본문 사이트 접근

            get_in_soup = BeautifulSoup(get_in_res.text, 'lxml')#들어간 본문안에서 HTML 긁어옴
            try:
                content = get_in_soup.find('td', {'class': 'board-contents'}).get_text(strip=True)
                content = re.sub(pattern, ' ', content)
                content = content.replace('\n', ' ') #본문 전체 긁어오고 개행문자 제거
            except:
                content = None
            csv_writer.writerow([title, link, date, view, content])
            print(title, link, date, view, content)
            


